In [2]:
from ctypes import *
libfm = CDLL("libfm_index.so")
libc = CDLL("libc-2.12.so")
printf = libc.printf

In [3]:
def load_index(fn):
    index = c_void_p()
    err = libfm.load_index(fn.encode("utf-8"), byref(index))
    if err:
        raise Error("Error loading index: %d" % err)
    return index

In [4]:
index = load_index("/hpc/users/willir31/dna.50MB")

In [5]:
def extract(index, pos, num):
    text = c_char_p()
    read = c_int()
    err = libfm.extract(index, pos, pos + num - 1, byref(text), byref(read))
    if err:
        raise Error("Error extracting from index: %d" % err)
    return text.value[:num]

In [36]:
extract(index, 0, 10)

b'GATCAATGAG'

In [10]:
def locate(index, pattern):
    occs = pointer(c_int())
    num_occs = c_int()
    err = libfm.locate(index, pattern.encode("utf-8"), len(pattern), byref(occs), byref(num_occs))
    if err:
        raise Error("Error locating %s: %d" % (pattern, err))
    ret = []
    for i in range(num_occs.value):
        ret.append(occs[i])
    return ret, num_occs.value

In [11]:
locate(index, "AACCCGGGTT")

([20466271, 2334036, 1069262, 39613552], 4)

In [12]:
def count(index, pattern):
    num_occs = c_int()
    err = libfm.count(index, pattern.encode("utf-8"), len(pattern), byref(num_occs))
    if err:
        raise Error("Error locating %s: %d" % (pattern, err))
    return num_occs.value

In [13]:
count(index, "AACCCGGGTT")

4

In [30]:
def display(index, pattern, num_chars):
    text = c_char_p()
    lengths = pointer(c_int())
    num_occs = c_int()
    err = libfm.display(
        index, 
        pattern.encode("utf-8"), 
        len(pattern), 
        num_chars, 
        byref(num_occs),
        byref(text),
        byref(lengths)
    )
    if err:
        raise Error("Error locating %s: %d" % (pattern, err))
    texts = []
    idx = 0
    ls = []
    for i in range(num_occs.value):
        texts.append(text.value[idx:idx+lengths[i]])
        ls.append(lengths[i])
        idx += lengths[i]
    return num_occs.value, texts, ls

In [35]:
display(index, "AACCCGGGTT", 5)

(4,
 [b'AAAAGAACCCGGGTTTTATT',
  b'ATTTGAACCCGGGTTTGCAT',
  b'ACTTGAACCCGGGTTGCAGT',
  b'GCTTGAACCCGGGTTGGGTA'],
 [20, 20, 20, 20])

In [17]:
from os import path
from ctypes import *

class FMIndex:
    
    def __init__(self, fn, libfm_index_name="libfm_index.so"):
        fmi_path = fn if fn.endswith(".fmi") else "%s.fmi" % fn
        self.libfm = CDLL(libfm_index_name)
        if path.exists(fmi_path):
            self.load(fn)
        elif path.exists(fn):
            self.build(fn)
        else:
            raise IOError("Specify a path to build or load index from; %s and %s don't exist" % (fmi_path, fn))
    
    def build(self, fn, outfile=None, buckets1=16, buckets2=512, freq=64):
        outfile = outfile if outfile else fn
        self.libfm.fm_build_index(fn.encode('utf-8'), outfile.encode("utf-8"), buckets1, buckets2, freq)
        self.load(outfile)
    
    def load(self, fn):
        self._index = c_void_p()
        if fn.endswith(".fmi"):
            fn = fn[:-4]
        err = self.libfm.load_index(fn.encode("utf-8"), byref(self._index))
        if err:
            raise Error("Error loading index: %d" % err)
    
    def count(self, pattern):
        num_occs = c_int()
        err = self.libfm.count(self._index, pattern.encode("utf-8"), len(pattern), byref(num_occs))
        if err:
            raise Error("Error locating %s: %d" % (pattern, err))
        return num_occs.value

    def locate(self, pattern):
        occs = pointer(c_int())
        num_occs = c_int()
        err = self.libfm.locate(self._index, pattern.encode("utf-8"), len(pattern), byref(occs), byref(num_occs))
        if err:
            raise Error("Error locating %s: %d" % (pattern, err))
        ret = []
        for i in range(num_occs.value):
            ret.append(occs[i])
        return ret, num_occs.value
    
    def display(self, pattern, num_chars):
        text = c_char_p()
        lengths = pointer(c_int())
        num_occs = c_int()
        err = self.libfm.display(
            self._index, 
            pattern.encode("utf-8"), 
            len(pattern), 
            num_chars, 
            byref(num_occs),
            byref(text),
            byref(lengths)
        )
        if err:
            raise Error("Error locating %s: %d" % (pattern, err))
        texts = []
        idx = 0
        ls = []
        for i in range(num_occs.value):
            texts.append(text.value[idx:idx+lengths[i]])
            ls.append(lengths[i])
            idx += lengths[i]
        return num_occs.value, texts, ls
    
    def extract(self, pos, num):
        text = c_char_p()
        read = c_int()
        err = self.libfm.extract(self._index, pos, pos + num - 1, byref(text), byref(read))
        if err:
            raise Error("Error extracting from index: %d" % err)
        return text.value[:num]


In [12]:
fmi = FMIndex("/hpc/users/willir31/dna.50MB.fmi")

In [13]:
fmi._index

c_void_p(51569344)

In [15]:
fmi.count("AACCCGGGTT")

4

In [16]:
fmi.locate("AACCCGGGTT")

([20466271, 2334036, 1069262, 39613552], 4)

In [17]:
fmi.extract(0, 20)

b'GATCAATGAGGTGGACACCA'

In [18]:
fmi.display("AACCCGGGTT", 5)

(4,
 [b'AAAAGAACCCGGGTTTTATT',
  b'ATTTGAACCCGGGTTTGCAT',
  b'ACTTGAACCCGGGTTGCAGT',
  b'GCTTGAACCCGGGTTGGGTA'],
 [20, 20, 20, 20])

In [20]:
fmi2 = FMIndex("/hpc/users/willir31/dna.50MB")

In [21]:
fmi2.display("AACCCGGGTT", 5)

(4,
 [b'AAAAGAACCCGGGTTTTATT',
  b'ATTTGAACCCGGGTTTGCAT',
  b'ACTTGAACCCGGGTTGCAGT',
  b'GCTTGAACCCGGGTTGGGTA'],
 [20, 20, 20, 20])

In [ ]:
fm189 = FMIndex("/hpc/users/willir31/data/pt189/PT_189_Left_Ovary_RNA.accepted_hits.sort.coord.dna")

In [1]:
import pysam

In [3]:
bam = pysam.AlignmentFile("/demeter/scratch/datasets/pt189/PT_189_Left_Ovary_RNA.accepted_hits.sort.coord.bam", "rb")

In [4]:
reads = bam.fetch()

ValueError: fetch called on bamfile without index

In [14]:
def to_dna(bamfile, outfile=None):
    bam = pysam.AlignmentFile(bamfile, "rb")
    outfile = outfile if outfile else (bamfile + ".dna")
    with open(outfile, 'w') as of:
        for read in bam:
            of.write(read.seq)
            of.write("\n")

In [15]:
to_dna(
    "/demeter/scratch/datasets/pt189/PT_189_Left_Ovary_RNA.accepted_hits.sort.coord.bam",
    "/hpc/users/willir31/data/pt189/PT_189_Left_Ovary_RNA.accepted_hits.sort.coord.dna"
)